In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta

import time
import schedule


es = Elasticsearch('http://127.0.0.1:9200')

#키워드는 "빅데이터","빅데이타" 연관검색어는 1개만 넣는것이 좋음 (중복된 데이터가 수집됨) 
competitor=['카카오','삼성sds','lgcns','디리아','메가존클라우드','nshc',"cloudera","엘라스틱서치코리아","네이버","이지팜"]
bigdata_industry = ["빅데이터", "마이데이터", "가트너","DW", "데이터웨어하우스", "Migration", "ETL","EAI","ESB","cloud","AWS","GCP"]
bigdata_solution = ["splunk","apache kafka", "apache spark", "elasticsearch","zookeeper", "confluent","yarn","kibana","logstash","file beats","docker","kubernetes"]
keyword = [competitor , bigdata_industry , bigdata_solution]

now = datetime.now()
today = now.strftime('%Y-%m-%d')
yesterday = now - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d-%H-%M')

title_list = []
url_list = []
comp_list = []
thumbnail_list = []
text_list = []
day_list = []
keyword_list =[]
category_list = []
keyword = sum(keyword,[])


def printhello():
    
    for j in range(len(competitor)+len(bigdata_industry)+len(bigdata_solution)) :      

        response1 = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+ keyword[j]
        response2= '&sort=0&photo=0&field=0&pd=4&ds=' + today + '&de=' + yesterday + '&cluster_rank=27&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1d,a:all&start='

        response = requests.get(response1+response2+str(1))
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        articles = soup.select('#main_pack > section > div > div.group_news > ul > li')


        for article in articles:
            a_tag1 = article.select_one('.news_tit')

            title = a_tag1.text
            title_list.append(title.strip())


            url = a_tag1['href']
            url_list.append(url.strip())

            comp = article.select_one('a.info.press').text
            comp = comp.replace('언론사 선정', '')
            comp_list.append(comp.strip())

            text = article.select_one('a.api_txt_lines.dsc_txt_wrap').text
            text_list.append(text.strip())

            day_list.append(today)

            keyword_list.append(keyword[j])

            try:
                thumbnail = article.select_one('div > a > img')['src']
                thumbnail_list.append(thumbnail.strip())

            except:
                thumbnail_list.append(
                    'https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F366%2F2021%2F09%2F23%2F762207.jpg&type=ff264_180&expire=2&refresh=true')
            # 일부 기사는 이미지가 없는 경우가 있다. 따라서 이미지가 없는 경우 기본값을 설정해주었다.


            if (keyword[j] in competitor):
                category_list.append("competitor")

            elif (keyword[j] in bigdata_industry):
                category_list.append("bigdata_industry")

            elif (keyword[j] in bigdata_solution):
                category_list.append("bigdata_solution")
                 
            
    info = {'title':title_list, 'url':url_list, 'company':comp_list, 'contents':text_list, 'date':day_list, 'image':thumbnail_list ,'keyword':keyword_list, 'category': category_list}

    print(info)

schedule.every(1).minutes.do(printhello) #1분마다 실행
schedule.every().monday.at("00:10").do(printhello) #월요일 00:10분에 실행
schedule.every().day.at("10:30").do(printhello) #매일 10시 30분에  

while True:
    schedule.run_pending()
    time.sleep(1)

{'title': ['이재명 "네이버·카카오·쿠팡 등 \'온라인 임대료\' 투명하게 공개"', "카카오 기프티콘, '품절 환불' 추진...'차액 환불'은 안돼", '카카오 이모티콘 출시 10주년…10억 이상 누적 매출 이모티콘 92개', '카카오뱅크 "중신용 대출자 절반이 신용점수 ↑"', '카카오 T택시 평균연령 2.6세↓...MZ세대 기사비중 4.1%p↑', '[카카오 넥스트] 류영준의 다음 도전은 카카오 신성장동력 발굴', '포스코 ICT RPA-카카오워크 연동 서비스 시작', '카카오, 디지털카드 서비스 확대', '공정위 “온플법 대상 네이버·카카오·구글 등 18개로 축소”', "파격, 또 파격…네이버·카카오, '40대 젊은피'로 조직쇄신 시동건다", '파수, 마이데이터 사업 활성화에 따른 수혜 전망-하나', "[K-Biz 리뷰] 전기차 배터리 시장 '600조' 성장 전망(11월29일)", '[클릭 e종목]"파수, 내달 마이데이터 시행으로 데이터보안 매출 확대"', '"파수, 마이데이터 사업 활성화 수혜"', '파수, 마이데이터 사업 활성화에 따른 수혜 전망-하나금투', '[카카오 넥스트] 류영준의 다음 도전은 카카오 신성장동력 발굴', '커지는 딥페이크 보이스피싱 우려...경고 잇따라', '천안 아파트 10여곳 웰패트 해킹 의혹 불안감 차단 전력투구', '아파트 월패드 영상 유출, 이미 2017년부터 해킹 이슈…집 값 떨어진다며 쉬쉬', '[더벨]LG CNS, 내부 인사로 CSO 세대교체', '[C를 찾아서] 정원석 LG마그나 대표, 구광모 가속페달 밟는 ‘전장사업’ 주역', '[인사] LG CNS', "[더벨][CFO 워치/LG디스플레이]'재무개선 소방수' 서동희 전무, 부사장 '영전...", "메가존클라우드·ETRI, 멀티클라우드 기술 사업화 '맞손'", '메가존클라우드·ETRI, 멀티클라우드 사업화 협력', '마인즈랩 주가 15%↑...메가존클라우드와 AI 플랫폼 공동사업 협력', '레드엔젤\xa0·메가존클라우드, 연탄 기부·나눔 봉사 활동', 

KeyboardInterrupt: 